# Numerical Mathematics - Assignment 1

## Peder Brekke, Simen Nesland and Espen Bjørge Urheim

### TASK 1

#### a)
$A \in \mathbb{R}^{n\times n}$ is non-singular, so $A$ has an inverse, $A^{-1}$.

$$\kappa_2(A)=\|A\|_2  \|A^{-1}\|_2,  \|\cdot\|_2=\sqrt{\lambda_{max}(\cdot)}$$

$$\Rightarrow \|A^{-1}\|=\sqrt{\lambda_{max}((A^{-1})^TA^{-1})}=\sqrt{\lambda_{max}((AA^T)^{-1})}$$

$AA^T$ and $A^TA$ have the same nonzero eigenvalues ($A^TAx=\lambda x \rightarrow AA^T(Ax)=\lambda (Ax), \square$). Thus,

$$\|A^{-1}\|_2=\sqrt{\lambda_{max}((A^TA)^{-1})}$$

We know that the eigenvalues of an inverse matrix are $\lambda(A^{-1})=\frac{1}{\lambda(A)}$.

$$\Rightarrow \lambda_{max}((A^TA)^{-1}) = \frac{1}{\lambda_{min}(A^TA)}$$

$$
\kappa(A)=\kappa_2(A)=\|A\|_2 \cdot \|A^{-1}\|_2=\sqrt{\lambda_{max}(A^TA)}\cdot\frac{1}{\sqrt{\lambda_{min}(A^TA)}} = (\frac{\lambda_{max}}{\lambda_{min}})^{0.5}. \square
$$

#### b)

For an orthogonal matrix, $Q$,

$$
Q^TQ=QQ^T=I \Leftrightarrow Q^T=Q^{-1}
$$

This gives

$$
\kappa_2(Q)= \|Q\|_2 \|Q^{-1}\|_2=\|Q\|_2 \|Q^T\|_2=\sqrt{\lambda_{max}(Q^TQ)}\cdot \sqrt{\lambda_{max}(QQ^T)} = \sqrt{\lambda_{max}(I)\cdot \lambda_{max}(I)} = 1. \square
$$

#### c)

$$
\kappa_2(A)= 1 \Rightarrow \|A\|_2 \|A^{-1}\|_2 = (\frac{\lambda_{max}(A^TA)}{\lambda_{min}(A^TA)})^{0.5} = 1 \Rightarrow \lambda_{max}(A^TA) = \lambda_{min}(A^TA)
$$

Thus, all eigenvalues of $A^TA$ must be equal. $\square$

### TASK 2
#### a)

In [1]:
import numpy as np
import scipy.linalg

def mylu(A):
    A = np.copy(A)
    n = np.shape(A)[0]
    p = np.arange(n)
    for i in range(n-1):
        max_ind = np.where(abs(A)[:,i][i:] == np.amax(abs(A)[:,i][i:]))[0][0] + i
        p[[i, max_ind]] = p[[max_ind, i]]
        A[[i, max_ind]] = A[[max_ind, i]]
        for j in range(i+1, n):
            m = A[j][i]/A[i][i]
            A[j][i:] -= m*A[i][i:]
            A[j][i] = m

    return A, p

#### b)

In [2]:
def L_U(LU):
    """
    Additional function to split overwritten A-matrix into L and U
    LU: Overwritten A-matrix
    Returns: L (lower triangular with 1 on diagonal), U (upper triagonal)
    """
    n = np.shape(LU)[0]
    L = np.zeros(np.shape(LU))
    U = np.zeros(np.shape(LU))
    for i in range(n):
        for j in range(n):
            if j>=i:
                U[i][j] = LU[i][j]
            else:
                L[i][j] = LU[i][j]
            if j == i:
                L[i][j] = 1
    return L, U

def mylutest(A):
    """
    Checks if mylu() is equal to scipy's LU-factorization using np.allclose()
    Returns: True/False
    """
    L_sc = scipy.linalg.lu(A)[1]
    U_sc = scipy.linalg.lu(A)[2]
    L, U = L_U(mylu(A)[0])
    if (np.allclose(L,L_sc) and np.allclose(U,U_sc)):
        return True
    else:
        return False

In [3]:
A = np.array([[2, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8]], dtype = float)
mylutest(A)

True

#### c)

In [4]:
B = np.array([[2, 5, 10, 7], [5, 2, 4, 8], [7, 5, 10, 6], [5, 4, 8, 8]], dtype = float)
print(mylu(B)[0])

[[ 7.          5.         10.          6.        ]
 [ 0.28571429  3.57142857  7.14285714  5.28571429]
 [ 0.71428571 -0.44        0.          6.04      ]
 [ 0.71428571  0.12               nan         nan]]


C:\Users\Simen\AppData\Local\Temp/ipykernel_94480/733284697.py:13: RuntimeWarning: invalid value encountered in double_scalars
  m = A[j][i]/A[i][i]


Beacuse two of the column vectors of $A$ are linearly dependent (i.e. we don't have full rank), we end up dividing by zero. At one point the pivot element of the current row will be zero, and when calculating the multiplier we divide by zero.

### TASK 3
#### a)

In [5]:
def fwdsub(A,b):
    n = np.shape(A)[0]
    x = np.zeros(n)
    for i in range(n):
        tot = b[i]
        for j in range(i+1):
            tot -= x[j]*A[i][j]
        x[i] = tot/A[i][i]
    return x

#### b)

In [6]:
def bcksub(A,b):
    A = np.flip(A)
    b = np.flip(b)
    return np.flip(fwdsub(A, b))

#### c)

In [7]:
def permutation(p):
    """
    Converts permutation vector to permutation matrix
    """
    P = []
    n = len(p)
    for i in range(n):
        row = np.zeros(n)
        row[p[i]] = 1
        P.append(row)
    return np.array(P)

def my_solve(A, b):
    LU, P = mylu(A)
    P = permutation(P)
    L, U = L_U(LU)
    y = fwdsub(L, P@b)
    x = bcksub(U, y)
    return x

In [8]:
# Testing previous code:

A = np.array([[2, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8]], dtype = float)
b = np.random.rand(1,4)[0]

print(my_solve(A, b))
print(scipy.linalg.solve(A,b))

[ 0.01554201  0.1445774  -0.00026109 -0.02918739]
[ 0.01554201  0.1445774  -0.00026109 -0.02918739]


#### d)

In [9]:
import time

def my_solve_LU(L, U, P, b):
    """
    Solves equation given L, U already computed
    """
    y = fwdsub(L, P@b)
    x = bcksub(U, y)
    return x

def testRuntime():
    A = np.random.rand(100,100)
    bs = np.random.rand(200,100)

    times = []

    # LU every time
    t0 = time.time()
    for i in range(200):
        x = my_solve(A, bs[i])
    times.append(time.time()-t0)

    # LU once
    t0 = time.time()
    LU, P = mylu(A)
    P = permutation(P)
    L, U = L_U(LU)
    for i in range(200):
        x = my_solve_LU(L, U, P, bs[i])
    times.append(time.time()-t0)

    # Scipy
    t0 = time.time()
    for i in range(200):
        x = np.linalg.solve(A, bs[i])
    times.append(time.time()-t0)

    return times

tests = ["Compute LU every time", "Compute LU once", "Scipy method"]
times = testRuntime()

for i, time in enumerate(times):
    print((tests[i] + ":").ljust(25), round(time,4), "sec")

Compute LU every time:    8.0681 sec
Compute LU once:          1.0877 sec
Scipy method:             0.017 sec


The method computing $L$ and $U$ only once is obviously way faster than the one that decomposes A every time. LU decomposition has time complexity $O(n^3)$ so computing this only once saves a lot of time. Scipy is faster beacuse the time-critical loops are written in other languages such as C, C++ and Fortran. Also, our algorithms may be improved slightly in terms of memory use and use of slow bulit-in functions.